<a href="https://colab.research.google.com/github/uzeziogho/Kaggle-competitions/blob/main/Rent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_selection import RFE
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
houstst=pd.read_csv('/content/test (1).csv')
houstrn=pd.read_csv('/content/train (1).csv')
rePrice=pd.read_csv('/content/sample_submission.csv')

In [5]:
houstst.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Prepare data
y_train = houstrn['SalePrice']
X_train = houstrn.drop(columns='SalePrice')

X_test= houstst
y_test= rePrice['SalePrice']

In [14]:
# handle missing values
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
dftrn_num = houstrn.select_dtypes(include=['float', 'int'])
dftrn_num = pd.DataFrame(imputer.fit_transform(dftrn_num), columns=dftrn_num.columns)

dftst_num= houstst.select_dtypes(include=['float','int'])
dftst_num=pd.DataFrame(imputer.fit_transform(dftst_num),columns=dftst_num.columns)

# handle categorical variables
dftrn_cat = houstrn.select_dtypes(include=['object'])
dftst_cat = houstst.select_dtypes(include=['object'])
le = LabelEncoder()
dftrn_cat = dftrn_cat.apply(le.fit_transform)
dftst_cat = dftst_cat.apply(le.fit_transform)

# combine numerical and categorical columns
df_encoded = pd.concat([dftrn_num, dftrn_cat], axis=1)
dfst_encoded = pd.concat([dftst_num,dftst_cat],axis=1)

# perform feature selection using RFE
estimator = RandomForestRegressor()
selector = RFE(estimator, n_features_to_select=50)
selector = selector.fit(df_encoded, houstrn['SalePrice'])
df_selected = df_encoded[df_encoded.columns[selector.support_]]

selector =selector.fit(dfst_encoded,y_test)
dfst_selected = dfst_encoded[dfst_encoded.columns[selector.support_]]

# train Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(df_selected, y_train)

# make predictions on test set
y_pred = rf.predict(dfst_selected)

# evaluate performance
from sklearn.metrics import mean_absolute_error

print("Mean Absolute Error: ", mean_absolute_error(y_test, y_pred))

Mean Absolute Error:  140158.50517206624


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- BldgType
- BsmtFinType2
- Condition1
- EnclosedPorch
- HeatingQC
- ...
Feature names seen at fit time, yet now missing:
- ExterQual
- FullBath
- LandSlope
- SalePrice
- SaleType
- ...

  warnings.warn(message, FutureWarning)


In [15]:
print(y_pred)

[38564.57 38907.24 39195.44 ... 38460.65 39151.29 39102.62]


In [18]:
class4=houstst[['Id']]

class4 = class4.assign(SalePrice=y_pred)

class4.head()

,Id,SalePrice
0,1461,38564.57
1,1462,38907.24
2,1463,39195.44
3,1464,38964.66
4,1465,38600.06
